In [1]:
import numpy as np
from matplotlib import pyplot as plt

#from skimage.transform import rescale, resize, radon, iradon
#from scipy.stats import mode

from utils2 import *
import h5py
from time import time

/home/everett/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# All

In [2]:
data = np.load('./sinograms/tomo_00064_to_00067/tomo_00066.npy')
sinogram = data[:,0,:]
shape = sinogram.shape[0]
start = time()
rowsum = np.sum(sinogram, axis=1)
window = shape//2
shifting_range = shape-window

bins = []
for i in range(shifting_range):
    bins.append(window_bins(rowsum, window, i, fbins = [1])[0])
    
root, spline = spline_root(np.imag(bins), window)
end = time()
print(root, end-start)

[151.46697972] 0.00993204116821289


# Load Data.

Center finder works on 2D sinograms so select a sinogram along the y axis and load the data as (x, theta)

In [3]:
#h_sino = h5py.File('./sinograms/tomo_00001/tomo_00001.h5', 'r')
data = np.load('./sinograms/tomo_00064_to_00067/tomo_00066.npy')
#h_sino = h5py.File('./phantoms/phantom_00002/phantom_00002.h5','r')
#data, theta = tomobank_phantom_sino('./phantoms/phantom_00008/phantom_00008.h5')
#print(data.shape)
#print(np.max(theta))

# Center finding on data and data with simulated.

In [4]:
## Define exposure and bit_depth for simulated data (enter None for real data)
exposure = 5
bit_depth = None

## Choose to sum along Y axis or to select a slice 
sinogram = data[:,0,:] #select a slice
#image_center = sinogram.shape[0]/2

## Add noise
#sinogram_noise, sinogram_noise_snr = add_noise2(sinogram, exposure, bit_depth = bit_depth)

## Sum along theta
rowsum = np.sum(sinogram, axis=1)
#rowsum_n = np.sum(sinogram_noise, axis=1)

## Print contrast ratio of arrays
## Contrast ratio is defined as the ratio of the max difference pixel values to
## the difference between minimum and maximum allowed values for the arrays dtype

#print('Contrast ratio of input data:           ',  contrast_ratio(sinogram))
#print('Contrast ratio of low exposure data:    ', contrast_ratio(sinogram_noise), '\n')
#print('Contrast ratio of input row sum:        ',  contrast_ratio(rowsum))
#print('Contrast ratio of low exposure row sum: ', contrast_ratio(rowsum_n))

## Plot ##
#imimplot(sinogram, sinogram_noise, (rowsum, rowsum_n),
#         ('Input Data', 'Exposure = ' + str(exposure)), title = 'Row Sum')

# Select window size, shifting range. Find center

In [5]:
## Define scan area and range
window = sinogram.shape[0]//2
shifting_range = sinogram.shape[0]-window

## Collect the 1st FFT bin
bins = []
#bins_n = []
for i in range(shifting_range):
    bins.append(window_bins(sinogram, window, i, fbins = [1])[0])
    #bins_n.append(window_bins(sinogram_noise, window, i, fbins = [1])[0])

## Fit curve to find zero crossing and thus center.
root, spline = spline_root(np.imag(bins), window)
#root_n, spline_n = spline_root(np.imag(bins_n), window)

## Find mode of the minimum of the real component of first spectrum
#fsm = first_spectrum_min(sinogram, window)
#fsm_n = first_spectrum_min(sinogram_noise, window)
#fsm_mode = mode(fsm)[0][0]
#fsm_n_mode = mode(fsm_n)[0][0]
print(root)

'''
## Plot results
plt.figure(figsize=(10,4))
plt.subplot(121)
plt.plot(fsm, label = 'Input')
plt.plot(fsm_n, label = 'Low Exposure')
plt.axhline(fsm_mode, color='g', linewidth = .5, label = 'Mode: Input')
plt.axhline(fsm_n_mode, color='r', linewidth = .5, label = 'Mode: Low Exposure')
plt.title('Mode of the minimum of first spectrum = ' + str(fsm_mode))
plt.legend()
plt.subplot(122)
plt.plot(np.arange(len(np.imag(bins)))*2,np.imag(bins)/np.max(np.imag(bins)), label = 'Input')
plt.plot(np.arange(len(np.imag(bins_n)))*2,np.imag(bins_n)/np.max(np.imag(bins_n)), label = 'Low Exposure')
plt.title('Odd Spectrum root = ' + str(root[0])[0:6])
plt.axhline(0, color = 'r', linewidth = .5, label = 'y = 0.00')
plt.legend()
plt.show()

plt.figure(figsize=(15,15))
plt.subplot(121)
plt.imshow(sinogram)
plt.title('Sinogram and center line')
plt.axhline(root, color='w', linewidth = 1, label = str(root[0])[0:6])
plt.legend()
plt.subplot(122)
plt.imshow(sinogram_noise)
plt.title('Sinogram and center line')
plt.axhline(root, color='w', linewidth = 1, label = str(root_n[0])[0:6])
plt.legend()
plt.show()
'''


[151.46697972]


"\n## Plot results\nplt.figure(figsize=(10,4))\nplt.subplot(121)\nplt.plot(fsm, label = 'Input')\nplt.plot(fsm_n, label = 'Low Exposure')\nplt.axhline(fsm_mode, color='g', linewidth = .5, label = 'Mode: Input')\nplt.axhline(fsm_n_mode, color='r', linewidth = .5, label = 'Mode: Low Exposure')\nplt.title('Mode of the minimum of first spectrum = ' + str(fsm_mode))\nplt.legend()\nplt.subplot(122)\nplt.plot(np.arange(len(np.imag(bins)))*2,np.imag(bins)/np.max(np.imag(bins)), label = 'Input')\nplt.plot(np.arange(len(np.imag(bins_n)))*2,np.imag(bins_n)/np.max(np.imag(bins_n)), label = 'Low Exposure')\nplt.title('Odd Spectrum root = ' + str(root[0])[0:6])\nplt.axhline(0, color = 'r', linewidth = .5, label = 'y = 0.00')\nplt.legend()\nplt.show()\n\nplt.figure(figsize=(15,15))\nplt.subplot(121)\nplt.imshow(sinogram)\nplt.title('Sinogram and center line')\nplt.axhline(root, color='w', linewidth = 1, label = str(root[0])[0:6])\nplt.legend()\nplt.subplot(122)\nplt.imshow(sinogram_noise)\nplt.title(

# Plot the rowsum, fft[1] spectrum, and its even and odd components.

## This was for playing around with different spectra

In [46]:
import math
def goertzel(samples, *freqs):
    """
    Adapted from author: Sebastien Piquemal https://github.com/sebpiq
    Implementation of the Goertzel algorithm, useful for calculating individual
    terms of a discrete Fourier transform.
    """
    window_size = len(samples)
    f_step = 1
    f_step_normalized = 1.0 / window_size

    # Calculate all the DFT bins we have to compute to include frequencies
    # in `freqs`.
    bins = set()
    for f_range in freqs:
        f_start, f_end = f_range
        k_start = int(math.floor(f_start / f_step))
        k_end = int(math.ceil(f_end / f_step))

        if k_end > window_size - 1: raise ValueError('frequency out of range %s' % k_end)
        bins = bins.union(range(k_start, k_end))

    # For all the bins, calculate the DFT term
    n_range = range(0, window_size)
    results = []
    for k in bins:

        # Bin frequency and coefficients for the computation
        f = k * f_step_normalized
        w_real = 2.0 * math.cos(2.0 * math.pi * f)
        w_imag = math.sin(2.0 * math.pi * f)

        # Doing the calculation on the whole sample
        d1, d2 = 0.0, 0.0
        for n in n_range:
            y  = samples[n] + w_real * d1 - d2
            d2, d1 = d1, y

        # Storing results `(real part, imag part, power)`
        results.append(w_imag * d1)
    return results

In [48]:
results = goertzel(rowsum, (1, 2))
freqfft = np.fft.rfft(rowsum)[1]
print(freqfft)
print(results)

(468063984.5436595+223885670.76535282j)
[223885670.7653974]
